In [ ]:
import pandas as pd

In [ ]:
from rdkit import Chem

In [ ]:
from tqdm import tqdm

In [ ]:
from IPython.display import display, SVG

In [ ]:
from utils import draw_reaction_smarts

In [ ]:
from regioselectivity_check import regioselectivity_check

In [ ]:
from site_prediction import ReactionSitePredictor

In [ ]:
tqdm.pandas()

In [ ]:
data = pd.read_csv("data/rxns_with_templates_hdfs_and_jsons.csv")

In [ ]:
site_pred = ReactionSitePredictor("models/LGBM_measured_allData_final_model.txt")

In [ ]:
data = data.drop_duplicates(subset=['rxn'], keep='first')

In [ ]:
data

In [ ]:
data["template"].nunique()

In [ ]:
ix = iter(range(len(data)))

In [ ]:
i = next(ix)
rxn = data["rxn"].iloc[i]
templ = data["template"].iloc[i]
print(templ)
print(rxn)
display(draw_reaction_smarts(templ, use_smiles=False))
display(draw_reaction_smarts(rxn, use_smiles=True))
regioselectivity_check(rxn, templ, site_pred)

In [ ]:
subset = data.iloc[30000:]

In [ ]:
subset

In [ ]:
def f(rxn, tpl, site_pred=site_pred):
    try:
        return regioselectivity_check(rxn, tpl, site_pred)
    except Exception as e:
        return ""

In [ ]:
data["decision"] = data.progress_apply(lambda x: f(x[0], x[1], site_pred), axis=1)

In [ ]:
data

In [ ]:
# def __unsubstituted_part_of_an_aromatic_ring(atom, mol):
#     for a in mol.GetAtoms():
#         if a.GetAtomMapNum() == atom.GetAtomMapNum():
#             return tuple([(i.GetIsAromatic() or "&a" in i.GetSmarts()) for i in a.GetNeighbors()]) == (True, True)


# def _template_relevant(template_left: str, template_right: str):
#     if "Br" in template_right and "Br" in template_left:
#         # Check if the bromine atom is connected to an aromatic carbon atom
#         left = Chem.MolFromSmarts(template_left)
#         right = Chem.MolFromSmarts(template_right)
#         for a in right.GetAtoms():
#             if a.GetSymbol() == "Br":
#                 hal_neighbor = a.GetNeighbors()[0]
#                 return hal_neighbor.GetAtomicNum() == 6 and hal_neighbor.GetIsAromatic() and __unsubstituted_part_of_an_aromatic_ring(hal_neighbor, left)
#     return False

In [ ]:
data["decision"].value_counts(normalize=True)

In [ ]:
data[["rxn", "template", "decision"]].to_csv("data/rxns_with_templates_hdfs_jsons_decisions.csv", index=False, header=True)

In [ ]:
data[data["decision"] == "undecided"]

In [ ]:
data = pd.concat((data, pd.get_dummies(data["decision"])), axis=1)

In [ ]:
data

In [ ]:
template_correctness = data.groupby(by="template")[["correct", "undecided", "wrong"]].mean().reset_index()

In [ ]:
template_correctness.head(2)

In [ ]:
template_correctness["correct"].hist(bins=100)

In [ ]:
template_correctness["undecided"].hist(bins=100)

In [ ]:
template_correctness["wrong"].hist(bins=100)

In [ ]:
ix = iter(template_correctness[template_correctness["wrong"] == 1].index)
# ix = iter(template_correctness[template_correctness["wrong"] == 1].index)

In [ ]:
i = next(ix)
t = template_correctness.loc[i]["template"]
print(t)
print(data[data.template == t]["rxn"].iloc[0])
# print(_template_relevant(*t.split(">>")))
display(draw_reaction_smarts(t, use_smiles=False))
draw_reaction_smarts(data[data.template == t]["rxn"].iloc[0])

In [ ]:
m = Chem.MolFromSmarts("[#7;a:1]:[c:2]:[c:3]:[cH;D2;+0:4]:[c;H0;D3;+0:6]")
display(m)
m.UpdatePropertyCache()
Chem.GetSymmSSSR(m)
print(Chem.MolToSmarts(m))
# m = Chem.MolFromSmarts(Chem.MolToSmarts(Chem.MolFromSmarts("[#7;a:1]:[c:2]:[c:3]:[cH;D2;+0:4]:[c;H0;D3;+0:6]")))
for i in m.GetAtoms():
    print(i.GetSmarts())
    print(dir(i))
    print(i.GetSymbol(), i.GetIsAromatic(), i.GetPropsAsDict())

In [ ]:
data[data.template == t]["rxn"]

In [ ]:
m = "[#7;a:2]:[c:3]:[c;H0;D3;+0:4](:[#7;a:5])-[Br;H0;D1;+0:1]"
mol = Chem.MolFromSmarts(m)
for a in mol.GetAtoms():
    if a.GetSymbol() == "Br":
        print(a.GetNeighbors()[0].GetIsAromatic())

In [ ]:
display(SVG('test_mol.svg'))